## Todo

- [x] Use `test_image.tif` and `solar_shapfile_test.shp` as input
- [x] Create mask image from both inputs
  - [x] Check the rasterized mask image
- [x] tiling on both `test_image.tif` and `rasterized_mask` for Segmentation task
- [x] Add padding before save `mask_tile.tif`

In [1]:
import rioxarray
import geopandas as gpd
import shapely as shp
import rasterio
from geo_rasterizer import rasterize
import os
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm

In [2]:
image_path = "../data/Sample1/image/test_image.tif"
shape_path = "../data/Sample1/shapefile/solar_shapefile_test.shp"
mask_path = "../data/Sample1/mask/mask.tif"

In [3]:
def create_mask(image_path:str, shape_path:str, output_path:str):
    # Load shape
    if(os.path.exists(shape_path) == False):
        raise FileExistsError(f"{shape_path=} is not exist")
    if(os.path.exists(image_path) == False):
        raise FileExistsError(f"{image_path=} is not exist")
    
    shape = gpd.read_file(shape_path)

    # Load DatasetReader
    ds_reader = rasterio.open(image_path)
    # create raster mask image
    mask = rasterize(shape, ds_reader)
    del(ds_reader)
    del(shape)
    
    ##### TODO #####
    ##### Shape might be bigger than input image ######

    # Save mask.tif
    image = rioxarray.open_rasterio(image_path)
    ds = xr.Dataset({
        'mask':xr.DataArray(
            data=mask,
            dims = ['latitude','longitude'],
            coords = {'latitude': image.y.values ,'longitude':image.x.values}
        )
    })
    ds.rio.write_crs('epsg:32647', inplace=True)
    ds.rio.to_raster(output_path, compress='LZW')
    del(mask)
    del(ds)
    del(image)

create_mask(image_path=image_path, shape_path=shape_path, output_path=mask_path)

In [19]:
import multiprocessing as mp
from itertools import product
# Tiling with 512 x 512 window size

def padding(tile:xr.DataArray, size:int) -> xr.DataArray:
    channel,y,x = tile.shape
    pad_x = size - x
    pad_y = size - y
    tile = tile.pad({'x':(0,pad_x), 'y':(0,pad_y)})
    return tile

def _tiling(data:tuple):
    
    img_type:str = data[0]
    img:xr.DataArray = data[1]
    tile_size:int = data[2]
    index_y:int = data[3]
    index_x:int = data[4]
    tile = img[:, index_y:index_y+tile_size , index_x:index_x+tile_size ]
    tile = padding(tile=tile, size=tile_size)
    if(img_type == 'image' and (3,tile_size,tile_size) != tile.shape):
        print(img_type, tile.shape)
    if(img_type == 'mask' and (1,tile_size,tile_size) != tile.shape):
        print(img_type, tile.shape)
    tile.rio.to_raster(f"../data/Sample1_tile/{img_type}/{img_type}_{index_y}_{index_x}.tif")

def _create_tile(data:tuple):
    img_type:str = data[0]
    img_path:str = data[1]
    tile_size:int = data[2]
    img:xr.DataArray = rioxarray.open_rasterio(img_path)
    for index_y in tqdm(range(0,len(img.y),tile_size)):
        for index_x in range(0,len(img.x),tile_size):
            tile = img[:, index_y:index_y+tile_size , index_x:index_x+tile_size ]
            tile = padding(tile=tile, size=tile_size)

            if(img_type == 'image' and (3,tile_size,tile_size) != tile.shape):
                print(img_type, tile.shape)
            if(img_type == 'mask' and (1,tile_size,tile_size) != tile.shape):
                print(img_type, tile.shape)

            tile.rio.to_raster(f"../data/Sample1_tile/{img_type}/{img_type}_{index_y}_{index_x}.tif")

def create_tile(image_path:str, mask_path:str, tile_size:int):
    if(os.path.exists(image_path) == False):
        raise FileExistsError(f"{image_path=} is not exist")
    if(os.path.exists(mask_path) == False):
        raise FileExistsError(f"{mask_path=} is not exist")
    
    if(tile_size % 32 != 0):
        raise ValueError(f"U-Net tile size should divisible by 32 but got {tile_size=}")

    with mp.Pool(processes=2) as pool:
        task_1 = ('image', image_path, tile_size)
        task_2 = ('mask', mask_path, tile_size)
        _ = [_ for _ in pool.imap_unordered(_create_tile, [task_1, task_2])]

# Add elapse time log
create_tile(image_path=image_path, mask_path=mask_path, tile_size=512)